<a href="https://colab.research.google.com/github/v-artur/Golden_Oreos/blob/main/speaker_indep_bigru_conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Obtaining the data

In [ ]:
#features
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1vtZchVzl424pSQBXQ8EBxvVOzcEQPKIp' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1vtZchVzl424pSQBXQ8EBxvVOzcEQPKIp" -O features.zip && rm -rf /tmp/cookies.txt
#original electrode names
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1A2CMLYAMOjET7Bdwt8bjRt8YLQeoVP80' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1A2CMLYAMOjET7Bdwt8bjRt8YLQeoVP80" -O subject_channels.zip && rm -rf /tmp/cookies.txt

# Data extraction
import zipfile
zip_ref = zipfile.ZipFile("/content/features.zip", 'r')
zip_ref.extractall("/content/features")
zip_ref.close()

# Electrode name extraction
zip_ref = zipfile.ZipFile("/content/subject_channels.zip", 'r')
zip_ref.extractall("/content")
zip_ref.close()


--2022-12-09 07:21:10--  https://docs.google.com/uc?export=download&confirm=t&id=1vtZchVzl424pSQBXQ8EBxvVOzcEQPKIp
Resolving docs.google.com (docs.google.com)... 108.177.125.102, 108.177.125.139, 108.177.125.113, ...
Connecting to docs.google.com (docs.google.com)|108.177.125.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/usphsnju7cn0bgk6897l018i9lc74i2u/1670570400000/17895932938140350971/*/1vtZchVzl424pSQBXQ8EBxvVOzcEQPKIp?e=download&uuid=2190d4b1-aec8-4159-a55a-5ddb1b93a41a [following]
--2022-12-09 07:21:11--  https://doc-08-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/usphsnju7cn0bgk6897l018i9lc74i2u/1670570400000/17895932938140350971/*/1vtZchVzl424pSQBXQ8EBxvVOzcEQPKIp?e=download&uuid=2190d4b1-aec8-4159-a55a-5ddb1b93a41a
Resolving doc-08-9o-docs.googleusercontent.com (doc-08-9o-docs.googleusercontent.com)... 64.233.188.1

<h3>Preparations and needed functions</h3>

Our plan is to transform every feature vector into a larger dimensional feature vector, which contains all the different electrode names across all the subjects. In the original features, every feature vector consisted of 9 smaller feature vectors which corresponded to the transformed iEEG signals across 9 consecutive timesteps, so every feature vector had some sort of sequentiality within itself. So in order to use BiGRU and Convolutional models effectively, we need to be specific about the ordering of our features.


In [ ]:
import pandas as pd
import numpy as np


# Getting the original electrode names
original_electrodes = set()

for subject in ['01','02','03','04','05','06','07','08','09','10']:
  table = pd.read_csv(f'/content/subject_channels/sub-{subject}_task-wordProduction_channels.tsv', sep='\t')
  elecs = set(table['name'])
  original_electrodes = original_electrodes.union(elecs)


# Now indexing them from -4 to 4 in order, so all features vectors will consist of 9 smaller feature vectors which will correspond to the
# 9 consecutive timesteps in the larger dimensional vectors (every step will have 540 channels)
all_electrodes = []
for i in range(9):
  for elec in original_electrodes:
    all_electrodes.append(elec + "T" + str(i-4))


print('Number of different features:', len(all_electrodes))

#we will use this list's indexes to correspond to the feature matrices
all_electrodes = list(all_electrodes) 

Number of different features: 4860


In [ ]:
# Function for making every feature vector into a 4860 dimensional vector

def dim_adjust(data, feature_names):
  #create a new matrix with zeros, insert the values into the columns
  #which correspond to the subject's feature names, then concatenate them

  new_matrix = np.zeros((data.shape[0],len(all_electrodes)))
  for column in range(data.shape[1]):
    insert_index = all_electrodes.index(feature_names[column])
    new_matrix[:,insert_index] = data[:,column]
    
  return new_matrix


## Making the iterated test, validation and test sets

In [ ]:
import os

feat_path = "/content/features"

# Function to generate the train, val and test features and mel spectrograms
def generate_features_and_spec():
  #Base arrays
  train_feat = np.empty((0, len(all_electrodes)))
  val_feat = np.empty((0, len(all_electrodes)))
  test_feat = np.empty((0, len(all_electrodes)))

  train_spec = np.empty((0, 23))
  val_spec = np.empty((0, 23))
  test_spec = np.empty((0, 23))

  for index, subject in enumerate(['01','02','03','04','05','06','07','08','09','10']):
    #loading the features, feature names and mel spectrogram of the subject
    data = np.load(os.path.join(feat_path,f'sub-{subject}_feat.npy'))
    feature_names = np.load(os.path.join(feat_path,f'sub-{subject}_feat_names.npy'))
    spectrogram = np.load(os.path.join(feat_path,f'sub-{subject}_spec.npy'))

    #splittig the features and the labels into 5 parts
    feat_splits = np.array_split(data, 5)
    spec_splits = np.array_split(spectrogram, 5)

    #making the train, val or test arrays for the subject using a 60-20-20 ratio
    #because of the "index" changing, the splitting position will iterate with each subject as well
    subject_train_feat = np.vstack((feat_splits[index % 5],feat_splits[(index+1) % 5],feat_splits[(index+2) % 5]))
    subject_train_spec = np.vstack((spec_splits[index % 5],spec_splits[(index+1) % 5],spec_splits[(index+2) % 5]))

    subject_val_feat = feat_splits[(index+3) % 5]
    subject_val_spec = spec_splits[(index+3) % 5]

    subject_test_feat = feat_splits[(index+4) % 5]
    subject_test_spec = spec_splits[(index+4) % 5]

    #concatenating the dimensionality-adjusted features with the pre-existing feature set
    train_feat = np.concatenate((train_feat, dim_adjust(subject_train_feat, feature_names)))
    val_feat = np.concatenate((val_feat, dim_adjust(subject_val_feat, feature_names)))
    test_feat = np.concatenate((test_feat, dim_adjust(subject_test_feat, feature_names)))

    #concatenating the appropriate arrays with the pre-existing train, val or test labels
    train_spec = np.concatenate((train_spec, subject_train_spec))
    val_spec = np.concatenate((val_spec, subject_val_spec))
    test_spec = np.concatenate((test_spec, subject_test_spec))

  return train_feat, train_spec, val_feat, val_spec, test_feat, test_spec 

# Generating the data
train_feat, train_spec, val_feat, val_spec, test_feat, test_spec = generate_features_and_spec()

In [ ]:
from sklearn.preprocessing import StandardScaler
# Scaling the data 

scaler = StandardScaler()
scaler.fit(train_feat)
train_feat = scaler.transform(train_feat)
val_feat = scaler.transform(val_feat)
test_feat = scaler.transform(test_feat)

In [ ]:
# Reshaping the data for the sequential models

train_feat = np.reshape(train_feat, (train_feat.shape[0], 9, int(train_feat.shape[1]/9)))
val_feat = np.reshape(val_feat, (val_feat.shape[0], 9, int(val_feat.shape[1]/9)))
test_feat = np.reshape(test_feat, (test_feat.shape[0], 9, int(test_feat.shape[1]/9)))

# Modeling

<h3>Defining the data generators</h3>

In [ ]:
from tensorflow.keras.utils import Sequence, set_random_seed

set_random_seed(1234)

class DataGenerator(Sequence):
    # Initialization
    def __init__(self, data, spec, batch_size=32, dim=(9, 540), shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.data = data
        self.shuffle = shuffle
        self.spec = spec
        self.on_epoch_end()

    def __len__(self):
        # Shows the number of batches per epoch
        return int(np.floor(self.data.shape[0] / self.batch_size))

    def __getitem__(self, index):
        # Generate one batch of data
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        X_batch = np.empty((self.batch_size, self.dim[0], self.dim[1]))
        y_batch = np.empty((self.batch_size, 23))
        for index1, elem in enumerate(indexes):
          X_batch[index1] = self.data[index1]
          y_batch[index1] = self.spec[index1]

        return X_batch, y_batch

    def on_epoch_end(self):
        # Updating the index after each epoch
        self.indexes = np.arange(self.data.shape[0])
        if self.shuffle == True:
            np.random.shuffle(self.indexes)


# Creating the generators
train_gen = DataGenerator(train_feat, train_spec, 256)
val_gen = DataGenerator(val_feat, train_spec, 256)
test_gen = DataGenerator(test_feat, train_spec, 256)

<h3>Tuning the GRU model</h3>

In [ ]:
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Input, Dropout, Bidirectional, GRU, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import keras_tuner as kt

def create_bigru_optimal(hp):

  model = tf.keras.Sequential()
  model.add(Input(shape=(9, 540)))

  # Recurrent layer 1
  hp_units_1 = hp.Int('units_1', min_value=128, max_value=512, step=64)
  hp_dropout_1 = hp.Float('dropout_1', min_value = 0.0, max_value = 0.5, default = 0.2, step = 0.1)
  model.add(Bidirectional(GRU(units=hp_units_1, return_sequences=True, dropout=hp_dropout_1)))

  # Recurrent layer 2
  hp_units_2 = hp.Int('units_2', min_value=64, max_value=256, step=32)
  hp_dropout_2 = hp.Float('dropout_2', min_value = 0.0, max_value = 0.5, default = 0.2, step = 0.1)
  model.add(Bidirectional(GRU(units=hp_units_2, return_sequences=True, dropout=hp_dropout_2)))

  # Recurrent layer 3
  hp_units_3 = hp.Int('units_3', min_value=16, max_value=64, step=8)
  hp_dropout_3 = hp.Float('dropout_3', min_value = 0.0, max_value = 0.5, default = 0.2, step = 0.1)
  model.add(Bidirectional(GRU(units=hp_units_3, return_sequences=True, dropout=hp_dropout_3)))

  # Final layers
  model.add(Flatten())
  model.add(Dense(23, activation = 'linear'))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mse',
                metrics=['mse'])
  
  return model


In [ ]:
# Setting up the tuner for the BiGRU model
tuner_bigru = kt.Hyperband(create_bigru_optimal,
                     objective='val_loss',
                     max_epochs=20,
                     factor = 3,
                     seed = 42,
                     directory='/content/bigru_opt',
                     project_name='bigru_opt1')

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Note: takes about 20 minutes to optimize
stop_early = EarlyStopping(monitor='val_loss', patience=10)
tuner_bigru.search(train_gen, epochs=100, validation_data=val_gen, verbose=0, shuffle=True, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps_bigru=tuner_bigru.get_best_hyperparameters(num_trials=1)[0]

<h3>Tuning the Convolutional model</h3>

In [ ]:
def create_conv_optimal(hp):
  model = tf.keras.Sequential()

  # 1st block
  hp_filter_1 = hp.Int('filter_1', min_value=16, max_value=64, step=8)
  hp_kernel_1 = hp.Choice('kernel_1', values=[3,4,5])
  hp_activation_1 = hp.Choice('activation_1', values=['relu', 'leakyrelu', 'swish'])
  model.add(Conv1D(filters=hp_filter_1, kernel_size=hp_kernel_1, activation=hp_activation_1, input_shape=(9, 540)))
  model.add(MaxPooling1D(pool_size=2))

  # 2nd block
  hp_filter_2 = hp.Int('filter_2', min_value=32, max_value=128, step=16)
  hp_kernel_2 = hp.Choice('kernel_2', values=[3,4,5])
  hp_activation_2 = hp.Choice('activation_1', values=['relu', 'leakyrelu', 'swish'])
  model.add(Conv1D(filters=hp_filter_2, kernel_size=hp_kernel_2, activation=hp_activation_2))
  model.add(MaxPooling1D(pool_size=2))

  model.add(Flatten())
  model.add(Dense(units=23, activation='linear'))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mse',
                metrics=['mse'])
  
  return model



In [ ]:
# Setting up the tuner for the convolutional model
tuner_conv = kt.Hyperband(create_conv_optimal,
                     objective='val_loss',
                     max_epochs=20,
                     factor = 3,
                     seed = 42,
                     directory='/content/conv_opt',
                     project_name='conv_opt1')

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Note: takes about 20 minutes to optimize
stop_early = EarlyStopping(monitor='val_loss', patience=10)
tuner_conv.search(train_gen, epochs=100, validation_data=val_gen, verbose=0, shuffle=True, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps_conv=tuner_conv.get_best_hyperparameters(num_trials=1)[0]